# Probing Many-Body Dynamics on Pulser

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip

from pulser import Pulse, Sequence, Register
from pulser.waveforms import CompositeWaveform, RampWaveform, ConstantWaveform
from pulser.devices import Chadoq2, Mock
from pulser.simulation import Simulation

In [ ]:
# Parameters in MHz and ns

delta_0 = -20 * 2*np.pi
delta_f = 20 * 2*np.pi
Omega_max = 2 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 200
t_stop = 10000
t_end = 10010

In [ ]:
R_blockade = (5.008e6/Omega_max)**(1/6)
reg = Register.rectangle(1,13, spacing=5.6, prefix='q')
print(f'Blockade Radius is: {R_blockade}µm.')
reg.draw()

## Z2 order: Creating the Pulse Sequence

In [ ]:
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

In [ ]:
seq = Sequence(reg, Mock)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

print(seq)
#seq.draw()

## Simulation

(Running with `nsteps = 1000`, `max_step=20`)

In [ ]:
simul = Simulation(seq)
print(simul._U, Omega_max, simul._U/Omega_max)

In [ ]:
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

In [ ]:
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect

In [ ]:
for x in res:
    plt.plot(x)

In [ ]:
def heat_detuning(data,start,end):
    time_window = []
    x =[]
    detunings = simul.samples['Global']['ground-rydberg']['det']

    for t,d in enumerate(detunings):
        if start <= d <= end:
            time_window.append(t)
            x.append(d)

    y = np.arange(len(reg.qubits))

    X, Y = np.meshgrid(x, y)
    Z = np.array(data)[:,time_window]

    plt.figure(figsize=(20,3))
    plt.pcolormesh(X,Y,Z, cmap='hot', shading='auto')

    plt.show()

In [ ]:
heat_detuning(res,delta_0,delta_f)

# Rydberg Crystals: Z3 Order

In [ ]:
reg = Register.rectangle(1,13, spacing=3.5, prefix='q')
reg.draw()

delta_0 = -20 * 2*np.pi
delta_f = 20 * 2*np.pi
Omega_max = 2.0 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 200
t_stop = 10000
t_end = 10010

#
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

#
seq = Sequence(reg, Mock)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

#print(seq)
#seq.draw()

simul = Simulation(seq)

#
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

#
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect

plt.figure(figsize=(10,5))
for expv in simul.output.expect:
    plt.plot(expv)
    
heat_detuning(res,delta_0,delta_f)
    
plt.show()

# Rydberg Crystals: Z4 Order

In [ ]:
reg = Register.rectangle(1,13, spacing=2.5, prefix='q')
reg.draw()

# Parameters in MHz and ns
#U = 1 * 2*np.pi  # btw 1-3 Mhz*2pi. Should we include this VdW magnitude in an AFM-type Simulation?

delta_0 = -10 * 2*np.pi
delta_f = 10 * 2*np.pi
Omega_max = 2.0 * 2*np.pi  # btw 1.8-2 Mhz*2pi
t_rise = 100
t_stop = 3000
t_end = 3010

#
hold = ConstantWaveform(t_rise, delta_0)
excite = RampWaveform(t_stop - t_rise, delta_0, delta_f)
sweep = Pulse.ConstantAmplitude(Omega_max, CompositeWaveform(hold, excite), 0.)
stay = Pulse.ConstantPulse(t_end - t_stop, 0., delta_f, 0.)

#
seq = Sequence(reg, Mock)
seq.declare_channel('ising', 'rydberg_global')

seq.add(sweep, 'ising')
seq.add(stay, 'ising')

#print(seq)
#seq.draw()

simul = Simulation(seq)

#
up = qutip.basis(2,0)
def occupation(j):
    prod = [qutip.qeye(2) for _ in range(simul._size)]
    prod[j] = up*up.dag()
    return qutip.tensor(prod)
    
occup_list = [occupation(j) for j in range(simul._size)]

#
simul.run(obs_list=occup_list, progress_bar=True)
res = simul.output.expect
        
heat_detuning(res,delta_0,delta_f)
    
#
plt.figure(figsize=(10,5))
for expv in simul.output.expect:
    plt.plot(expv)
    
plt.show()